In [ ]:
# The preliminary notebook for testing/developing wrsamp. Will delete before merging. 

## Function structure

### wrsamp(sig, fields)



- wrsamp will fill in default values for: nsig, siglen, basetime, fmt, if digital: (gain, baseline, units), . When a default value is filled in, print a warning message which states whether it was filled in because it was missing or because a requirement. 


Seems very difficult to define a 'isrequired' field in a class. Very convoluted. 



- Call wrheader(). At this point, the checksum, initvalue, res, nsig, siglen, fmt  have to already be calculated/present. 
  
  
- wrheader WILL check for validity of fields, and for the presence of (valid) requirements. We will not let it write invalid headers. It will NOT fill in default values, neither the calculated nor the non calculated ones. Summary: wrheader makes NO assumptions. 





recfields = OrderedDict([('recordname', WFDBfield(['recordname', [str], '', None, True, True, None])),
                         ('nseg', WFDBfield(['nseg', [int], '/', 'recordname', False, False, None])),
                         ('nsig', WFDBfield(['nsig', [int], ' ', 'recordname', True, True, None])),
                         ('fs', WFDBfield(['fs', [int, float], ' ', 'nsig', False, True, None])),
                         ('counterfreq', WFDBfield(['counterfreq', [int, float], '/', 'fs', False, False, None])),
                         ('basecounter', WFDBfield(['basecounter', [int, float], '(', 'counterfreq', False, False, None])),
                         ('siglen', WFDBfield(['siglen', [int], ' ', 'fs', False, True, None])),
                         ('basetime', WFDBfield(['basetime', [str], ' ', 'siglen', False, False, None])),
                         ('basedate', WFDBfield(['basedate', [str], ' ', 'basetime', False, False, None]))])

sigfields = OrderedDict([('filename', WFDBfield(['filename', [str], '', None, True, True, None])),
                         ('fmt', WFDBfield(['fmt', [int, str], ' ', 'filename', True, True, None])),
                         ('sampsperframe', WFDBfield(['sampsperframe', [int], 'x', 'fmt', False, False])),
                         ('skew', WFDBfield(['skew', [int], ':', 'fmt', False, False])),
                         ('byteoffset', WFDBfield(['byteoffset', [int], '+', 'fmt', False, False])),
                         ('adcgain', WFDBfield(['adcgain', [int], ' ', 'fmt', False, False])),
                         ('baseline', WFDBfield(['baseline', [int], '(', 'adcgain', False, False])),
                         ('units', WFDBfield(['units', [str], '/', 'adcgain', False, False])),
                         ('adcres', WFDBfield(['adcres', [int], ' ', 'adcgain', False, False])),
                         ('adczero', WFDBfield(['adczero', [int], ' ', 'adcres', False, False])),
                         ('initvalue', WFDBfield(['initvalue', [int], ' ', 'adczero', False, False])),
                         ('checksum', WFDBfield(['checksum', [int], ' ', 'initvalue', False, False])),
                         ('blocksize', WFDBfield(['blocksize', [int], ' ', 'checksum', False, False])),
                         ('signame', WFDBfield(['signame', [str], ' ', 'blocksize', False, False]))])
    

segfields = OrderedDict([('segname', WFDBfield(['segname', [str], '', None, True, True])),
                         ('seglen', WFDBfield(['seglen', [int], ' ', 'segname', True, True]))])

comfields = OrderedDict([('comments', WFDBfield(['comments', [int], '', None, False, False]))])



In [ ]:
from collections import OrderedDict
fields = OrderedDict([('height', 170), ('weight', 70), ('age', 20)])

for f in fields:
    print(f)

In [8]:
from collections import OrderedDict

# The frisking will not be done here, but rather in wrsamp and wrheader. 
# Single segment wfdb record.
class WFDBrecord():
    
    def __init__(self, recordname=None, nseg=1, nsig=None, 
                 fs=None, counterfreq=None, basecounter=None, 
                 siglen=None, basetime=None, basedate=None, 
                 filename=None, fmt=None, sampsperframe=None, 
                 skew=None, byteoffset=None, adcgain=None, 
                 baseline=None, units=None, adcres=None, 
                 adczero=None, initvalue=None, checksum=None, 
                 blocksize=None, signame=None, comments=None):
        
        if nseg>1:
            sys.exit("The WFDBrecord class is for single segment "
                     "records. Use the 'WFDBmultirecord' class for "
                     "multi-segment records.")
        
        self.recordname=recordname
        self.nseg=nseg
        self.nsig=nsig
        self.fs=fs
        self.counterfreq=counterfreq
        self.basecounter=basecounter
        self.siglen=siglen
        self.basetime=basetime
        self.basedate=basedate
        
        self.filename=filename
        self.fmt=fmt
        self.sampsperframe=sampsperframe
        self.skew=skew
        self.byteoffset=byteoffset
        self.adcgain=adcgain
        self.baseline=baseline
        self.units=units
        self.adcres=adcres
        self.adczero=adczero
        self.initvalue=initvalue
        self.checksum=checksum
        self.blocksize=blocksize
        self.signame=signame
        
        self.comments=comments
        
# Multi segment WFDB record. Stores a list of individual WFDBrecord objects in the 'segment' field.
class WFDBmultirecord():
     def __init__(self, recordname=None, nseg=None, nsig=None, 
                  fs=None, counterfreq=None, basecounter=None,
                  siglen=None, basetime=None, basedate=None,
                  seglen=None, segname=None, segment=None, 
                  comments=None):
        
        if nseg<2:
            sys.exit("The WFDBmultirecord class is for multi-segment "
                     "records. Use the 'WFDBrecord' class for "
                     "single segment records.")
    
        self.recordname=recordname
        self.nseg=nseg
        self.fs=fs
        self.counterfreq=counterfreq
        self.basecounter=basecounter
        self.siglen=siglen
        self.basetime=basetime
        self.basedate=basedate
        
        self.seglen=seglen
        self.segname=segname
        self.segment=sement
        
        self.comments=comments
        
# item:
# 0. type
# 1. delimiter
# 2. dependency
# 3. required to write? (nseg starts as F but will be set by wrmultiheader and wrmultiseg to True.)
# 4. default fill function
# A field of the WFDB header file
class WFDBfield():
    
    def __init__(self, speclist):
    
        # Data types the field can take
        self.allowedtypes = speclist[0]
        
        # The text delimiter that preceeds the field
        self.delimiter = speclist[1]
        
        # The required/dependent field which must also be present
        self.dependency = speclist[2]
        
        # Whether the field is mandatory for writing (WFDB requirements + extra rules enforced by this library).
        # Being required for writing is not the same as the user having to specify via wrsamp. wrsamp sets defaults (wrheader does not).  
        self.write_req = speclist[3]
        
        # The default function to call to obtain the default. Only called via wrsamp, not wrhea.
        # If 'None', it means there is nothing to call, not that 'None' is is the default value of the field. 
        
        # Or perhaps there should only be one function to call with if statements. Kind of inappropriate to 
        # define a bunch of lambda functions with various input requirements. Just redefine the values once for 
        # setdefault. In this case, this field will just be true or false whether there is a potential default 
        # from wrsamp (there is none for wrhea)
        # Watch out for: adcgain, baseline, units. Set them to true but have a check statement in the function. 
        self.has_write_default = speclist[4]
        
        
recfields = OrderedDict([('recordname', WFDBfield([[str], '', None, True, False])),
                         ('nseg', WFDBfield([[int], '/', 'recordname', False, False])),
                         ('nsig', WFDBfield([[int], ' ', 'recordname', True, True])),
                         ('fs', WFDBfield([[int, float], ' ', 'nsig', True, False])),
                         ('counterfreq', WFDBfield([[int, float], '/', 'fs', False, False])),
                         ('basecounter', WFDBfield([[int, float], '(', 'counterfreq', False, False])),
                         ('siglen', WFDBfield([[int], ' ', 'fs', True, True])),
                         ('basetime', WFDBfield([[str], ' ', 'siglen', False, True])),
                         ('basedate', WFDBfield([[str], ' ', 'basetime', False, False]))])

sigfields = OrderedDict([('filename', WFDBfield([[str], '', None, True, True])),
                         ('fmt', WFDBfield([[int, str], ' ', 'filename', True, True])),
                         ('sampsperframe', WFDBfield([[int], 'x', 'fmt', False, False])),
                         ('skew', WFDBfield([[int], ':', 'fmt', False, False])),
                         ('byteoffset', WFDBfield([[int], '+', 'fmt', False, False])),
                         ('adcgain', WFDBfield([[int], ' ', 'fmt', False, True])),
                         ('baseline', WFDBfield([[int], '(', 'adcgain', False, True])),
                         ('units', WFDBfield([[str], '/', 'adcgain', False, True])),
                         ('adcres', WFDBfield([[int], ' ', 'adcgain', False, True])),
                         ('adczero', WFDBfield([[int], ' ', 'adcres', False, True])),
                         ('initvalue', WFDBfield([[int], ' ', 'adczero', False, True])),
                         ('checksum', WFDBfield([[int], ' ', 'initvalue', False, True])),
                         ('blocksize', WFDBfield([[int], ' ', 'checksum', False, True])),
                         ('signame', WFDBfield([[str], ' ', 'blocksize', False, True]))])
    

segfields = OrderedDict([('segname', WFDBfield([[str], '', None, True, False])),
                         ('seglen', WFDBfield([[int], ' ', 'segname', True, False]))])

comfields = OrderedDict([('comments', WFDBfield([[int], '', None, False, False]))])


        
        
        
        
        
        

In [10]:
import warnings
warnings.warn('logfile argument deprecated', DeprecationWarning)

/home/chen/Software/pythonenvs/python3.5/lib/python3.5/site-packages/ipykernel/__main__.py:2: UserWarning: logfile argument deprecated
  from ipykernel import kernelapp as app


In [ ]:
bitlevels=np.power(2, np.array(range(1, 33)))

nlevels=13000

bitlevels<nlevels
print(bitlevels)
np.argmax(bitlevels>13000)

In [ ]:
WFDBfields = ['height', 'weight', 'age']

final = dict.fromkeys(WFDBfields, [])
final

In [ ]:
help(dict.fromkeys)

In [ ]:
for i in reversed(WFDBfields):
    print(i)

In [ ]:
a=[{'recname': 'wave1'},{'siglen': 500},{'fs': 125}]

In [ ]:
# A field of the WFDB header file
from collections import OrderedDict

class WFDBfield():
    
    def __init__(self, speclist):
        # Name
        self.name = speclist[0] 
        
        # Data types the field can take
        self.allowedtypes = speclist[1]
        
        # The text delimiter that preceeds the field
        self.delimiter = speclist[2]
        
        # The required/dependent field which must also be present
        self.dependency = speclist[3]

        # Whether the field is mandatory specified by the WFDB guidelines
        self.is_req = speclist[4]
        
        # Whether the field is mandatory for writing (extra rules enforced by this library).
        # Being required for writing is not the same as the user having to specify. There are defaults. 
        self.write_req = speclist[5]

In [ ]:
# 1. Record specification fields
recfields = OrderedDict([('recordname', WFDBfield(['recordname', [str], '', None, True, True, None])),
                         ('nseg', WFDBfield(['nseg', [int], '/', 'recordname', False, False, None])),
                         ('nsig', WFDBfield(['nsig', [int], ' ', 'recordname', True, True, None])),
                         ('fs', WFDBfield(['fs', [int, float], ' ', 'nsig', False, True, None])),
                         ('counterfreq', WFDBfield(['counterfreq', [int, float], '/', 'fs', False, False, None])),
                         ('basecounter', WFDBfield(['basecounter', [int, float], '(', 'counterfreq', False, False, None])),
                         ('siglen', WFDBfield(['siglen', [int], ' ', 'fs', False, True, None])),
                         ('basetime', WFDBfield(['basetime', [str], ' ', 'siglen', False, False, None])),
                         ('basedate', WFDBfield(['basedate', [str], ' ', 'basetime', False, False, None]))])
    
# 2. Signal specification fields
sigfields = OrderedDict([('filename', WFDBfield(['filename', [str], '', None, True, True, None])),
                         ('fmt', WFDBfield(['fmt', [int, str], ' ', 'filename', True, True, None])),
                         ('sampsperframe', WFDBfield(['sampsperframe', [int], 'x', 'fmt', False, False])),
                         ('skew', WFDBfield(['skew', [int], ':', 'fmt', False, False])),
                         ('byteoffset', WFDBfield(['byteoffset', [int], '+', 'fmt', False, False])),
                         ('adcgain', WFDBfield(['adcgain', [int], ' ', 'fmt', False, False])),
                         ('baseline', WFDBfield(['baseline', [int], '(', 'adcgain', False, False])),
                         ('units', WFDBfield(['units', [str], '/', 'adcgain', False, False])),
                         ('adcres', WFDBfield(['adcres', [int], ' ', 'adcgain', False, False])),
                         ('adczero', WFDBfield(['adczero', [int], ' ', 'adcres', False, False])),
                         ('initvalue', WFDBfield(['initvalue', [int], ' ', 'adczero', False, False])),
                         ('checksum', WFDBfield(['checksum', [int], ' ', 'initvalue', False, False])),
                         ('blocksize', WFDBfield(['blocksize', [int], ' ', 'checksum', False, False])),
                         ('signame', WFDBfield(['signame', [str], ' ', 'blocksize', False, False]))])
    
# 3. Segment specification fields
segfields = OrderedDict([('segname', WFDBfield(['segname', [str], '', None, True, True])),
                         ('seglen', WFDBfield(['seglen', [int], ' ', 'segname', True, True]))])
    
# 4. Comment fields
comfields = OrderedDict([('comments', WFDBfield(['comments', [int], '', None, False, False]))])


In [ ]:
WFDBfields=[recfields, sigfields, segfields, comfields]

[f for f in WFDBfields]

In [ ]:
f2=WFDBfields[1]

In [ ]:
f3=f1.copy()
f3.update(f2)

In [ ]:
import sys
sys.exit("Missing input fields['nseg'] for writing multi-segment header.\nFor writing regular single segment headers, use the 'wrheader' function.")

In [ ]:
# 1. Record specification fields
recfields = OrderedDict([('recordname', WFDBfield(['recordname', [str], '', None, True, True, None])),
                         ('nseg', WFDBfield(['nseg', [int], '/', 'recordname', False, False, None])),
                         ('nsig', WFDBfield(['nsig', [int], ' ', 'recordname', True, True, None])),
                         ('fs', WFDBfield(['fs', [int, float], ' ', 'nsig', False, True, None])),
                         ('counterfreq', WFDBfield(['counterfreq', [int, float], '/', 'fs', False, False, None])),
                         ('basecounter', WFDBfield(['basecounter', [int, float], '(', 'counterfreq', False, False, None])),
                         ('siglen', WFDBfield(['siglen', [int], ' ', 'fs', False, True, None])),
                         ('basetime', WFDBfield(['basetime', [str], ' ', 'siglen', False, False, None])),
                         ('basedate', WFDBfield(['basedate', [str], ' ', 'basetime', False, False, None]))])
    
# 2. Signal specification fields
sigfields = OrderedDict([('filename', WFDBfield(['filename', [str], '', None, True, True, None])),
                         ('fmt', WFDBfield(['fmt', [int, str], ' ', 'filename', True, True, None])),
                         ('sampsperframe', WFDBfield(['sampsperframe', [int], 'x', 'fmt', False, False])),
                         ('skew', WFDBfield(['skew', [int], ':', 'fmt', False, False])),
                         ('byteoffset', WFDBfield(['byteoffset', [int], '+', 'fmt', False, False])),
                         ('adcgain', WFDBfield(['adcgain', [int], ' ', 'fmt', False, False])),
                         ('baseline', WFDBfield(['baseline', [int], '(', 'adcgain', False, False])),
                         ('units', WFDBfield(['units', [str], '/', 'adcgain', False, False])),
                         ('adcres', WFDBfield(['adcres', [int], ' ', 'adcgain', False, False])),
                         ('adczero', WFDBfield(['adczero', [int], ' ', 'adcres', False, False])),
                         ('initvalue', WFDBfield(['initvalue', [int], ' ', 'adczero', False, False])),
                         ('checksum', WFDBfield(['checksum', [int], ' ', 'initvalue', False, False])),
                         ('blocksize', WFDBfield(['blocksize', [int], ' ', 'checksum', False, False])),
                         ('signame', WFDBfield(['signame', [str], ' ', 'blocksize', False, False]))])
    
# 3. Segment specification fields
segfields = OrderedDict([('segname', WFDBfield(['segname', [str], '', None, True, True])),
                         ('seglen', WFDBfield(['seglen', [int], ' ', 'segname', True, True]))])
    
# 4. Comment fields
comfields = OrderedDict([('comments', WFDBfield(['comments', [int], '', None, False, False]))])

In [ ]:
reqfields=OrderedDict()
for k, v in sigfields.items():
    if v.write_req:
        reqfields.update(OrderedDict([(k,v)]))


In [ ]:
reqfields

In [19]:
class man():
    def __init__(self):
        self.age=50
        self.height=150
        self.weight=80
    
    
kevin = man()
    
importantfields = ['height', 'weight']

for i in importantfields:
    print(getattr(kevin, i)) 

hasattr(kevin, 'height')

150
80


True

In [26]:
recfields = OrderedDict([('recordname', WFDBfield([[str], '', None, True, False])),
                         ('nseg', WFDBfield([[int], '/', 'recordname', False, False])), 
                         ('nsig', WFDBfield([[int], ' ', 'recordname', True, True])),
                         ('fs', WFDBfield([[int, float], ' ', 'nsig', True, False])),
                         ('counterfreq', WFDBfield([[int, float], '/', 'fs', False, False])),
                         ('basecounter', WFDBfield([[int, float], '(', 'counterfreq', False, False])),
                         ('siglen', WFDBfield([[int], ' ', 'fs', True, True])),
                         ('basetime', WFDBfield([[str], ' ', 'siglen', False, True])),
                         ('basedate', WFDBfield([[str], ' ', 'basetime', False, False]))])

sigfields = OrderedDict([('filename', WFDBfield([[str], '', None, True, True])),
                         ('fmt', WFDBfield([[int, str], ' ', 'filename', True, True])),
                         ('sampsperframe', WFDBfield([[int], 'x', 'fmt', False, False])),
                         ('skew', WFDBfield([[int], ':', 'fmt', False, False])),
                         ('byteoffset', WFDBfield([[int], '+', 'fmt', False, False])),
                         ('adcgain', WFDBfield([[int], ' ', 'fmt', False, True])),
                         ('baseline', WFDBfield([[int], '(', 'adcgain', False, True])),
                         ('units', WFDBfield([[str], '/', 'adcgain', False, True])),
                         ('adcres', WFDBfield([[int], ' ', 'adcgain', False, True])),
                         ('adczero', WFDBfield([[int], ' ', 'adcres', False, True])),
                         ('initvalue', WFDBfield([[int], ' ', 'adczero', False, True])),
                         ('checksum', WFDBfield([[int], ' ', 'initvalue', False, True])),
                         ('blocksize', WFDBfield([[int], ' ', 'checksum', False, True])),
                         ('signame', WFDBfield([[str], ' ', 'blocksize', False, True]))])
    

segfields = OrderedDict([('segname', WFDBfield([[str], '', None, True, False])),
                         ('seglen', WFDBfield([[int], ' ', 'segname', True, False]))])

comfields = OrderedDict([('comments', WFDBfield([[int], '', None, False, False]))])


WFDBfieldlist = [recfields.copy(), segfields.copy(), comfields.copy()]

def _mergeODlist(ODlist):
    mergedOD=ODlist[0].copy()
    for od in ODlist[1:]:
        mergedOD.update(od)
    return mergedOD

WFDBfields=_mergeODlist(WFDBfieldlist)

In [30]:
rev_rfields=OrderedDict(reversed(list(WFDBfieldlist[0].items())))

In [32]:
rev_rfields['basedate'].write_req

False